Instalação e importações necessárias

In [78]:
!pip install groq

In [79]:
import os
os.environ['GROQ_API_KEY'] = "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"

Testando o funcionamento da API do Groq

In [80]:
import os
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-70b-8192",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's natural language processing (NLP) landscape, and their importance can be seen in several aspects:

1. **Real-time Applications**: Fast language models enable real-time applications such as chatbots, virtual assistants, and language translation systems to respond quickly and efficiently. This is particularly important in customer-facing applications where delayed responses can lead to frustration and a negative user experience.
2. **Low-Latency Inference**: Fast language models can perform inference (i.e., make predictions or generate text) rapidly, which is essential for applications that require immediate responses, such as:
	* Sentiment analysis for social media monitoring
	* Text classification for spam detection
	* Language translation for real-time communication
3. **Scalability**: Fast language models can handle large volumes of data and scale to meet the demands of large-scale applications, such as:
	* Processing large datasets for dat

Criando a classe Agente

o método init recebe uma instância da API (cliente) e uma mensagem de sistema opcional, depois armazena ambos como atributos e cria uma lista para armazenar o histórico de mensagens

o método call recebe uma mensagem do usuário e adiciona ao histórico, chamando o método execute para obter uma resposta e, adiciona a resposta ao histórico de mensagem, retornando ela

por fim, o método execute envia o histórico de mensagens pra API do modelo, além de extrair e retornar o conteúdo da resposta gerada

In [81]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content

Definindo o Prompt

Eu adicionei novas funções (que serão explicadas na próxima célula) para aumentar a potencia do agente e, claro, para que eu tenha um aprendizado mais completo

In [82]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns mass of the planet in kg

get_planet_gravity:
e.g. get_planet_gravity: Mars
returns the surface gravity of the planet in m/s² and how it compares to Earth

get_planet_temperature:
e.g. get_planet_temperature: Mars
returns average, minimum and maximum temperature of the planet in Celsius

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

Observation: 1.1944e+25

Answer: The mass of Earth times 2 is 1.1944e+25 kg.

Now it's your turn:
""".strip()

Definindo as funções do Agente

a calculte receberá uma string de uma operação matemática e através da função e val vai retornar o resultado

já a get_planet_mass receberá o nome de um planeta como entrada e retornar a sua massa em kg, mas através de notação científica

In [83]:
def calculate(operation: str) -> float:
    return eval(operation)


def get_planet_mass(planet) -> float:
    match planet.lower():
        case "earth":
            return 5.972e24
        case "jupiter":
            return 1.898e27
        case "mars":
            return 6.39e23
        case "mercury":
            return 3.285e23
        case "neptune":
            return 1.024e26
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "venus":
            return 4.867e24
        case _:
            return 0.0

Funções novas adicionadas por mim

get_planet_gravity recebe o nome de um planeta e retorna a gravidade da sua superfície, em m/s² e compara com a da terra

get_planet_temperature recebe o nome de um planeta e retorna a temperatura mínima, máxima e média em graus Celsius

In [84]:
def get_planet_gravity(planet: str) -> str:
    gravity_data = {
        "mercury": {"gravity": 3.7, "compared_to_earth": "38% of Earth"},
        "venus": {"gravity": 8.87, "compared_to_earth": "90% of Earth"},
        "earth": {"gravity": 9.81, "compared_to_earth": "reference (100%)"},
        "mars": {"gravity": 3.71, "compared_to_earth": "38% of Earth"},
        "jupiter": {"gravity": 24.79, "compared_to_earth": "253% of Earth"},
        "saturn": {"gravity": 10.44, "compared_to_earth": "106% of Earth"},
        "uranus": {"gravity": 8.87, "compared_to_earth": "90% of Earth"},
        "neptune": {"gravity": 11.15, "compared_to_earth": "114% of Earth"}
    }

    planet = planet.lower().strip()
    if planet in gravity_data:
        data = gravity_data[planet]
        return f"Gravity in {planet.capitalize()}: {data['gravity']} m/s² ({data['compared_to_earth']})."
    else:
        return f"Gravity data not available for '{planet}'."


def get_planet_temperature(planet: str) -> str:
    planet_temps = {
        "mercury": {"min": -173, "max": 427, "avg": 167, "desc": "extremely variable due to lack of atmosphere"},
        "venus": {"min": 462, "max": 462, "avg": 462, "desc": "nearly uniform due to dense atmosphere"},
        "earth": {"min": -89, "max": 56, "avg": 15, "desc": "habitable, with variations by region and season"},
        "mars": {"min": -153, "max": 20, "avg": -63, "desc": "with large daily and seasonal variations"},
        "jupiter": {"min": -145, "max": -145, "avg": -145, "desc": "relatively constant in the upper layers"},
        "saturn": {"min": -178, "max": -178, "avg": -178, "desc": "cold and constant in the upper layers"},
        "uranus": {"min": -224, "max": -224, "avg": -224, "desc": "extremely cold and uniform"},
        "neptune": {"min": -218, "max": -218, "avg": -218, "desc": "very cold with occasional thunderstorms"}
    }

    planet = planet.lower().strip()
    if planet in planet_temps:
        data = planet_temps[planet]
        return f"Temperature for {planet.capitalize()}: Average of {data['avg']}°C (min: {data['min']}°C, max: {data['max']}°C). {data['desc'].capitalize()}."
    else:
        return f"No temperature data available for '{planet}'."

Execução do Agente com loop

a função agent_loop automatiza o ciclo, recebendo as iterações, o prompt e a pergunta inicial

o loop principal incrementa o contador de iterações, envia o prompt atual ao agente e imprime sua resposta se a resposta contém PAUSE ou ACTION, é extraído o nome da ferramenta e o argumento se ele for válido, executa a função e prepara a observação

caso a resposta contenha ANSWER, encerra o loop, visto que está definido que seria a resposta final

In [85]:
import re

def agent_loop(max_iterations, system, query):
    agent = Agent(client=client, system=system_prompt)
    tools = ["calculate", "get_planet_mass", "get_planet_gravity", "get_planet_temperature"]
    next_prompt = query
    i = 0
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break

In [86]:
agent_loop(10, system = system_prompt, query="If I weigh 75 kg on Earth, how much would I weigh on Mars and Jupiter?")

Thought: I need to find the surface gravity of Mars and Jupiter to calculate my weight on those planets.

Action: get_planet_gravity: Mars
PAUSE
Observation: Gravity in Mars: 3.71 m/s² (38% of Earth).
Thought: I now have the surface gravity of Mars, but I also need the surface gravity of Jupiter. 

Action: get_planet_gravity: Jupiter
PAUSE
Observation: Gravity in Jupiter: 24.79 m/s² (253% of Earth).
Thought: I now have the surface gravity of both Mars and Jupiter. I need to calculate my weight on these planets. 

Action: calculate: 75 * (3.71 / 9.81)
PAUSE
Observation: 28.363914373088686
Thought: I have now calculated my weight on Mars. I need to calculate my weight on Jupiter.

Action: calculate: 75 * (24.79 / 9.81)
PAUSE
Observation: 189.52599388379204
Thought: I have now calculated my weight on both Mars and Jupiter.

Answer: If I weigh 75 kg on Earth, I would weigh approximately 28.36 kg on Mars and approximately 189.53 kg on Jupiter.


In [87]:
agent_loop(10, system = system_prompt, query="What is the average temperature difference between Mars and Venus?")

Thought: I need to find the average temperatures of Mars and Venus.
Thought: I need to find the average temperatures of Mars and Venus.
Action: get_planet_temperature: Mars
PAUSE
Observation: Temperature for Mars: Average of -63°C (min: -153°C, max: 20°C). With large daily and seasonal variations.
Thought: Now I have the temperature of Mars. I need to get the temperature of Venus.
Action: get_planet_temperature: Venus
PAUSE
Observation: Temperature for Venus: Average of 462°C (min: 462°C, max: 462°C). Nearly uniform due to dense atmosphere.
Thought: Now I have the temperatures of both Mars and Venus. I need to calculate the difference in average temperatures.
Action: calculate: 462 - (-63)
PAUSE
Observation: 525
Thought: I have calculated the average temperature difference between Mars and Venus.
Answer: The average temperature difference between Mars and Venus is 525°C.
